# Start client

In [1]:
import dask.dataframe as dd
from dask.distributed import Client

#Initializing client
client = Client()
client

/anaconda3/envs/overscripted/lib/python3.6/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/anaconda3/envs/overscripted/lib/python3.6/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


Client Scheduler: tcp://127.0.0.1:50021 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 8.59 GB


# Data
Using 10% sample and self produced samples
 - 10% sample has 11292867 rows
 - Filtered by value_len > df.mean() has 499805 rows

In [3]:
#Original sample
df = dd.read_parquet('sample_0.parquet', 
                     engine='pyarrow', 
                     columns=['value_1000', 'value', 'value_len', 'symbol', 'script_url', 'location'])

# df.astype({'value_1000': str, 'value': str,'value_len': int,'symbol': int,'script_url': str})
df.columns

Index(['value_1000', 'value', 'value_len', 'symbol', 'script_url', 'location'], dtype='object')

### Filtered value_len > 1356
1356 is the value_len mean

To filter the data into something that is more interesting to this task I decided to only work with values that are at above the mean.

All values above the mean count up to 499805 rows. That is just 4,42% of the whole sample, and a lot easier to work on. 

In [4]:
#Save
df = df[df['value_len'] > 1356]
dd.to_parquet(df=df, path='filtered_above_mean.parquet', engine='pyarrow')
# len(dff)

In [7]:
#Read
df = dd.read_parquet('filtered_above_mean.parquet', engine='pyarrow')
df.columns

Index(['value_1000', 'value', 'value_len', 'symbol', 'script_url', 'location'], dtype='object')

## DF overview
Some overview about the sample: 
- Mean: 1356.97,
- Min: 0,
- Max: 4496861
- Std: 26310.62

In [9]:
df_mean = df['value_len'].mean()
df_min = df['value_len'].min()
df_max = df['value_len'].max()
df_std = df['value_len'].std()
(df_mean, df_min, df_max, df_std) = dd.compute(df_mean, df_min, df_max, df_std);
print(df_mean, df_min, df_max, df_std)

1356.9776628910975 0 4496861 26310.62140481331


# Domains
The following code is from this same project: ~/analyses/hello_world.ipynb


In [8]:
import tldextract

def extract_domain(url):
    """Use tldextract to return the base domain from a url"""
    try:
        extracted = tldextract.extract(url)
        return '{}.{}'.format(extracted.domain, extracted.suffix)
    except Exception as e:
        return 'ERROR'

In [10]:
df.astype({'value_1000': str, 'value': str,'value_len': int,'symbol': int,'script_url': str, 'location': str})
df['location_domain'] = df.location.apply(extract_domain)
df['script_domain'] = df.script_url.apply(extract_domain)

In [11]:
#save
df.to_parquet('0_sample_domains.parquet', engine='pyarrow')

In [12]:
#read
df = dd.read_parquet('0_sample_domains.parquet', engine='pyarrow')
df.head()

,value_1000,value,value_len,symbol,script_url,location,location_domain,script_domain
0,"{""im-settings"":""{\""val\"":{\""settings\"":{\""Site...","{""im-settings"":""{\""val\"":{\""settings\"":{\""Site...",3713,window.sessionStorage,https://assets.adobedtm.com/caacec67651710193d...,https://www.canada.ca/en/services.html,canada.ca,adobedtm.com
1,"{""APLUS_S_CORE_0.17.12_20171214163401_2ee09a0c...","{""APLUS_S_CORE_0.17.12_20171214163401_2ee09a0c...",103878,window.localStorage,https://g.alicdn.com/alilog/mlog/aplus_v2.js,https://maniform.world.tmall.com/category-1282...,tmall.com,alicdn.com
2,"{""APLUS_S_CORE_0.17.12_20171214163401_2ee09a0c...","{""APLUS_S_CORE_0.17.12_20171214163401_2ee09a0c...",103878,window.localStorage,https://g.alicdn.com/alilog/mlog/aplus_v2.js,https://maniform.world.tmall.com/category-1282...,tmall.com,alicdn.com
3,usunico=17/12/2017:0-00155123:830; SessionASM=...,usunico=17/12/2017:0-00155123:830; SessionASM=...,1358,window.document.cookie,https://www.coches.net/scripts/common.min.js?2...,https://www.coches.net/fiat/segunda-mano/,coches.net,coches.net
4,usunico=17/12/2017:0-00155123:830; SessionASM=...,usunico=17/12/2017:0-00155123:830; SessionASM=...,1358,window.document.cookie,https://www.coches.net/scripts/common.min.js?2...,https://www.coches.net/fiat/segunda-mano/,coches.net,coches.net


# is JSON Column

After manual initial analysis I have think that the huge values are json structured, to validate that I included an new column that is a boolean value with the validation of json

In [13]:
import json

def is_json(myjson):
    try:
        json.loads(myjson)
        return True
    except ValueError as e:
        return False

In [14]:

df['is_json'] = df['value'].apply(is_json)

### is_JSON data
Saving the new produced data with 'is_json' columns into disk

In [15]:
#save
df.to_parquet('is_json_above_mean.parquet')

/anaconda3/envs/overscripted/lib/python3.6/site-packages/fastparquet/util.py:221: FutureWarning: A future version of pandas will default to `skipna=True`. To silence this warning, pass `skipna=True|False` explicitly.
  inferred_dtype = infer_dtype(column)


In [8]:
#read
df = dd.read_parquet('is_json_above_mean.parquet')
df.head()

,value_1000,value,value_len,symbol,script_url,location,location_domain,script_domain,is_json
0,"{""im-settings"":""{\""val\"":{\""settings\"":{\""Site...","{""im-settings"":""{\""val\"":{\""settings\"":{\""Site...",3713,window.sessionStorage,https://assets.adobedtm.com/caacec67651710193d...,https://www.canada.ca/en/services.html,canada.ca,adobedtm.com,True
1,"{""APLUS_S_CORE_0.17.12_20171214163401_2ee09a0c...","{""APLUS_S_CORE_0.17.12_20171214163401_2ee09a0c...",103878,window.localStorage,https://g.alicdn.com/alilog/mlog/aplus_v2.js,https://maniform.world.tmall.com/category-1282...,tmall.com,alicdn.com,True
2,"{""APLUS_S_CORE_0.17.12_20171214163401_2ee09a0c...","{""APLUS_S_CORE_0.17.12_20171214163401_2ee09a0c...",103878,window.localStorage,https://g.alicdn.com/alilog/mlog/aplus_v2.js,https://maniform.world.tmall.com/category-1282...,tmall.com,alicdn.com,True
3,usunico=17/12/2017:0-00155123:830; SessionASM=...,usunico=17/12/2017:0-00155123:830; SessionASM=...,1358,window.document.cookie,https://www.coches.net/scripts/common.min.js?2...,https://www.coches.net/fiat/segunda-mano/,coches.net,coches.net,False
4,usunico=17/12/2017:0-00155123:830; SessionASM=...,usunico=17/12/2017:0-00155123:830; SessionASM=...,1358,window.document.cookie,https://www.coches.net/scripts/common.min.js?2...,https://www.coches.net/fiat/segunda-mano/,coches.net,coches.net,False


# Value md5
Include new columns called "value_md5" that is the md5 of value column

In [9]:
import hashlib

def md5(value):
    return hashlib.md5(value.encode('utf-8')).hexdigest()

In [10]:
df['value_md5'] = df['value'].apply(md5)

In [11]:
df.head()

,value_1000,value,value_len,symbol,script_url,location,location_domain,script_domain,is_json,value_md5
0,"{""im-settings"":""{\""val\"":{\""settings\"":{\""Site...","{""im-settings"":""{\""val\"":{\""settings\"":{\""Site...",3713,window.sessionStorage,https://assets.adobedtm.com/caacec67651710193d...,https://www.canada.ca/en/services.html,canada.ca,adobedtm.com,True,cff77029e3ae45dd439a62987b1d8340
1,"{""APLUS_S_CORE_0.17.12_20171214163401_2ee09a0c...","{""APLUS_S_CORE_0.17.12_20171214163401_2ee09a0c...",103878,window.localStorage,https://g.alicdn.com/alilog/mlog/aplus_v2.js,https://maniform.world.tmall.com/category-1282...,tmall.com,alicdn.com,True,9ac0a0a0afb677c8fd985a7c2f4ddbc5
2,"{""APLUS_S_CORE_0.17.12_20171214163401_2ee09a0c...","{""APLUS_S_CORE_0.17.12_20171214163401_2ee09a0c...",103878,window.localStorage,https://g.alicdn.com/alilog/mlog/aplus_v2.js,https://maniform.world.tmall.com/category-1282...,tmall.com,alicdn.com,True,9ac0a0a0afb677c8fd985a7c2f4ddbc5
3,usunico=17/12/2017:0-00155123:830; SessionASM=...,usunico=17/12/2017:0-00155123:830; SessionASM=...,1358,window.document.cookie,https://www.coches.net/scripts/common.min.js?2...,https://www.coches.net/fiat/segunda-mano/,coches.net,coches.net,False,db64465b639e01993d9212390f057628
4,usunico=17/12/2017:0-00155123:830; SessionASM=...,usunico=17/12/2017:0-00155123:830; SessionASM=...,1358,window.document.cookie,https://www.coches.net/scripts/common.min.js?2...,https://www.coches.net/fiat/segunda-mano/,coches.net,coches.net,False,db64465b639e01993d9212390f057628


In [ ]:
#save
df.to_parquet('is_json_above_mean_md5.parquet')

/anaconda3/envs/overscripted/lib/python3.6/site-packages/fastparquet/util.py:221: FutureWarning: A future version of pandas will default to `skipna=True`. To silence this warning, pass `skipna=True|False` explicitly.
  inferred_dtype = infer_dtype(column)
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
tornado.application - ERROR - Exception in callback <bound method Nanny.memory_monitor of <Nanny: tcp://127.0.0.1:50075, threads: 1>>
Traceback (most recent call last):
  File "/anaconda3/envs/overscripted/lib/python3.6/site-packages/psutil/_common.py", line 341, in wrapper
    ret = self._cache[fun]
AttributeError: 'Process' object has no attribute '_cache'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/anaconda3/envs/overscripted/lib/python3.6/site-packages/psutil/_common.py", line 341, in wrapper
    ret = self._cache[fun]
AttributeError: _cache

During handling of the above exception, anoth

tornado.application - ERROR - Exception in callback <bound method Nanny.memory_monitor of <Nanny: tcp://127.0.0.1:50070, threads: 1>>
Traceback (most recent call last):
  File "/anaconda3/envs/overscripted/lib/python3.6/site-packages/psutil/_common.py", line 341, in wrapper
    ret = self._cache[fun]
AttributeError: 'Process' object has no attribute '_cache'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/anaconda3/envs/overscripted/lib/python3.6/site-packages/psutil/_common.py", line 341, in wrapper
    ret = self._cache[fun]
AttributeError: _cache

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/anaconda3/envs/overscripted/lib/python3.6/site-packages/psutil/_psosx.py", line 359, in catch_zombie
    yield
  File "/anaconda3/envs/overscripted/lib/python3.6/site-packages/psutil/_psosx.py", line 400, in _get_pidtaskinfo
    ret = cext.proc_pidtaskinfo_oneshot

tornado.application - ERROR - Exception in callback <bound method Nanny.memory_monitor of <Nanny: tcp://127.0.0.1:59757, threads: 1>>
Traceback (most recent call last):
  File "/anaconda3/envs/overscripted/lib/python3.6/site-packages/psutil/_common.py", line 341, in wrapper
    ret = self._cache[fun]
AttributeError: 'Process' object has no attribute '_cache'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/anaconda3/envs/overscripted/lib/python3.6/site-packages/psutil/_common.py", line 341, in wrapper
    ret = self._cache[fun]
AttributeError: _cache

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/anaconda3/envs/overscripted/lib/python3.6/site-packages/psutil/_psosx.py", line 359, in catch_zombie
    yield
  File "/anaconda3/envs/overscripted/lib/python3.6/site-packages/psutil/_psosx.py", line 400, in _get_pidtaskinfo
    ret = cext.proc_pidtaskinfo_oneshot

In [16]:
#read
df = dd.read_parquet('is_json_above_mean_md5.parquet')
df.head()

,value_1000,value,value_len,symbol,script_url,location,location_domain,script_domain,is_json
0,"{""im-settings"":""{\""val\"":{\""settings\"":{\""Site...","{""im-settings"":""{\""val\"":{\""settings\"":{\""Site...",3713,window.sessionStorage,https://assets.adobedtm.com/caacec67651710193d...,https://www.canada.ca/en/services.html,canada.ca,adobedtm.com,True
1,"{""APLUS_S_CORE_0.17.12_20171214163401_2ee09a0c...","{""APLUS_S_CORE_0.17.12_20171214163401_2ee09a0c...",103878,window.localStorage,https://g.alicdn.com/alilog/mlog/aplus_v2.js,https://maniform.world.tmall.com/category-1282...,tmall.com,alicdn.com,True
2,"{""APLUS_S_CORE_0.17.12_20171214163401_2ee09a0c...","{""APLUS_S_CORE_0.17.12_20171214163401_2ee09a0c...",103878,window.localStorage,https://g.alicdn.com/alilog/mlog/aplus_v2.js,https://maniform.world.tmall.com/category-1282...,tmall.com,alicdn.com,True
3,usunico=17/12/2017:0-00155123:830; SessionASM=...,usunico=17/12/2017:0-00155123:830; SessionASM=...,1358,window.document.cookie,https://www.coches.net/scripts/common.min.js?2...,https://www.coches.net/fiat/segunda-mano/,coches.net,coches.net,False
4,usunico=17/12/2017:0-00155123:830; SessionASM=...,usunico=17/12/2017:0-00155123:830; SessionASM=...,1358,window.document.cookie,https://www.coches.net/scripts/common.min.js?2...,https://www.coches.net/fiat/segunda-mano/,coches.net,coches.net,False


# Saving other possible usefull samples to future analyses

In [19]:
df[df['is_json'] == True].to_parquet('all_json_above_mean.parquet', engine='pyarrow')

distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
tornado.application - ERROR - Exception in callback <bound method Nanny.memory_monitor of <Nanny: tcp://127.0.0.1:60669, threads: 1>>
Traceback (most recent call last):
  File "/anaconda3/envs/overscripted/lib/python3.6/site-packages/psutil/_common.py", line 341, in wrapper
    ret = self._cache[fun]
AttributeError: 'Process' object has no attribute '_cache'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/anaconda3/envs/overscripted/lib/python3.6/site-packages/psutil/_common.py", line 341, in wrapper
    ret = self._cache[fun]
AttributeError: _cache

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/anaconda3/envs/overscripted/lib/python3.6/site-packages/psutil/_psosx.py", line 359, in catch_zombie
    yield
  File "/anaconda3/envs/overscripted/lib/python3.6/site-packages/psutil/_psos

tornado.application - ERROR - Exception in callback <bound method Nanny.memory_monitor of <Nanny: tcp://127.0.0.1:60679, threads: 1>>
Traceback (most recent call last):
  File "/anaconda3/envs/overscripted/lib/python3.6/site-packages/psutil/_common.py", line 341, in wrapper
    ret = self._cache[fun]
AttributeError: 'Process' object has no attribute '_cache'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/anaconda3/envs/overscripted/lib/python3.6/site-packages/psutil/_common.py", line 341, in wrapper
    ret = self._cache[fun]
AttributeError: _cache

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/anaconda3/envs/overscripted/lib/python3.6/site-packages/psutil/_psosx.py", line 359, in catch_zombie
    yield
  File "/anaconda3/envs/overscripted/lib/python3.6/site-packages/psutil/_psosx.py", line 400, in _get_pidtaskinfo
    ret = cext.proc_pidtaskinfo_oneshot

psutil.AccessDenied: psutil.AccessDenied (pid=21484)
tornado.application - ERROR - Exception in callback <bound method Nanny.memory_monitor of <Nanny: tcp://127.0.0.1:60699, threads: 1>>
Traceback (most recent call last):
  File "/anaconda3/envs/overscripted/lib/python3.6/site-packages/psutil/_common.py", line 341, in wrapper
    ret = self._cache[fun]
AttributeError: 'Process' object has no attribute '_cache'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/anaconda3/envs/overscripted/lib/python3.6/site-packages/psutil/_common.py", line 341, in wrapper
    ret = self._cache[fun]
AttributeError: _cache

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/anaconda3/envs/overscripted/lib/python3.6/site-packages/psutil/_psosx.py", line 359, in catch_zombie
    yield
  File "/anaconda3/envs/overscripted/lib/python3.6/site-packages/psutil/_psosx.py", line 400, in _ge

tornado.application - ERROR - Exception in callback <bound method Nanny.memory_monitor of <Nanny: tcp://127.0.0.1:60690, threads: 1>>
Traceback (most recent call last):
  File "/anaconda3/envs/overscripted/lib/python3.6/site-packages/psutil/_common.py", line 341, in wrapper
    ret = self._cache[fun]
AttributeError: 'Process' object has no attribute '_cache'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/anaconda3/envs/overscripted/lib/python3.6/site-packages/psutil/_common.py", line 341, in wrapper
    ret = self._cache[fun]
AttributeError: _cache

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/anaconda3/envs/overscripted/lib/python3.6/site-packages/psutil/_psosx.py", line 359, in catch_zombie
    yield
  File "/anaconda3/envs/overscripted/lib/python3.6/site-packages/psutil/_psosx.py", line 400, in _get_pidtaskinfo
    ret = cext.proc_pidtaskinfo_oneshot

tornado.application - ERROR - Exception in callback <bound method Nanny.memory_monitor of <Nanny: tcp://127.0.0.1:60716, threads: 1>>
Traceback (most recent call last):
  File "/anaconda3/envs/overscripted/lib/python3.6/site-packages/psutil/_common.py", line 341, in wrapper
    ret = self._cache[fun]
AttributeError: 'Process' object has no attribute '_cache'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/anaconda3/envs/overscripted/lib/python3.6/site-packages/psutil/_common.py", line 341, in wrapper
    ret = self._cache[fun]
AttributeError: _cache

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/anaconda3/envs/overscripted/lib/python3.6/site-packages/psutil/_psosx.py", line 359, in catch_zombie
    yield
  File "/anaconda3/envs/overscripted/lib/python3.6/site-packages/psutil/_psosx.py", line 400, in _get_pidtaskinfo
    ret = cext.proc_pidtaskinfo_oneshot

tornado.application - ERROR - Exception in callback <bound method Nanny.memory_monitor of <Nanny: tcp://127.0.0.1:60707, threads: 1>>
Traceback (most recent call last):
  File "/anaconda3/envs/overscripted/lib/python3.6/site-packages/psutil/_common.py", line 341, in wrapper
    ret = self._cache[fun]
AttributeError: 'Process' object has no attribute '_cache'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/anaconda3/envs/overscripted/lib/python3.6/site-packages/psutil/_common.py", line 341, in wrapper
    ret = self._cache[fun]
AttributeError: _cache

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/anaconda3/envs/overscripted/lib/python3.6/site-packages/psutil/_psosx.py", line 359, in catch_zombie
    yield
  File "/anaconda3/envs/overscripted/lib/python3.6/site-packages/psutil/_psosx.py", line 400, in _get_pidtaskinfo
    ret = cext.proc_pidtaskinfo_oneshot

In [4]:
df[df['is_json'] == False].to_parquet('all_NON_json_above_mean.parquet')